In [3]:
#import tensorflow as tf
import numpy as np
import pandas as pd
import scipy as sp
from scipy.stats import chi2
from math import floor
from sklearn.svm import LinearSVC

In [266]:
#Initial data setup

X = pd.read_csv("/home/a20114261/GeneInteractions/GeneInteractionsBN_Datasets/Balanced_Batches/censored_Erk_Target/batch_0.csv")

# each cell can be -1, 0 or 1
#Pre-processing data
#X['Pvalue']=list(map(lambda x: 1 if x<=0.01 else (0 if x>=0.5 else -1), X['Pvalue']))
X['Pvalue']=list(map(lambda x: 1 if x<=0.05 else 0, X['Pvalue']))
del X['Unnamed: 0']
del X['TrueIndex']
del X['CauseGene']
del X['EffectGene']
del X['Replicate']
del X['Treatment']
PC=[]
for column in X:
    if (column != "Pvalue"):
        X[column] = list(map(lambda x: x+1 , X[column]))
x_heads = []
for i in range(0,len(X.keys())):
    max_v=1
    for row in X[X.keys()[i]]:
        if (row>max_v-1):
            max_v+=row-(max_v-1)
    save_i=i+1
    x_heads.append("Node"+str(save_i)+"@"+str(max_v))
X.columns = x_heads
data_c_size = len(X[X.keys()[1]])
data_vars = [0,1,2,3]

In [24]:
X = pd.read_csv("/home/a20114261/Alarm1_s5000_v1.txt",delimiter='  ',header=None)
x_heads = []
for i in range(0,len(X.keys())):
    max_v=1
    for row in X[X.keys()[i]]:
        if (row>max_v-1):
            max_v+=row-(max_v-1)
    save_i=i+1
    x_heads.append("Node"+str(save_i)+"@"+str(max_v))
X.columns = x_heads
data_c_size = len(X[X.keys()[1]])
data_vars = [0,1,2,3]

/home/a20114261/anaconda3/envs/GeneInteractionsBN/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


5997

In [ ]:
# Initial setup for TensorFlow
with tf.device('/job:localhost/replica:0/task:0/device:GPU:1'):
    a = tf.placeholder(tf.float32,shape=(data_c_size,))
    b = tf.placeholder(tf.float32,shape=(data_c_size,))
    dot_mult_op = tf.einsum('i,i->',a, b)
    
    c = tf.placeholder(tf.float32,shape=(data_c_size,))
    d = tf.placeholder(tf.float32,shape=(data_c_size,))
    var_mask = tf.equal(c,d)
    
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.50)
    sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
                 


In [ ]:
def tf_const_vector(n,size):
    global sess
    with tf.device('/job:localhost/replica:0/task:0/device:GPU:1'):
        a = tf.constant(np.zeros(size),tf.float32)
        b = tf.constant(n,tf.float32)
        v_constructor = a+b
        return sess.run(v_constructor)

In [ ]:
def tf_dot_mult(arr1,arr2):
    global sess
    global dot_mult_op
    return sess.run(dot_mult_op,{a:arr1,b:arr2})

In [ ]:
def tf_mask_var(arr1,var):
    global sess
    global var_mask
    global const_vectors
    return sess.run(var_mask,{c:arr1,d:const_vectors[var]})

In [ ]:
def tf_mask_equal(arr1,arr2):
    global sess
    global var_mask
    return sess.run(var_mask,{c:arr1,d:arr2})

In [ ]:
def setup_const_vectors(var_arr,size):
    returnable_dict = {}
    for i in range(len(var_arr)):
        returnable_dict[var_arr[i]] = tf_const_vector(var_arr[i],size)
    
    return returnable_dict

In [4]:
def ExistDseparator(TargetNode,Xi, Z, X, alpha):
    flagExist = False
    dsepSet=[]
    #counter=0
    #print_names(Z)
    for i in range(0,(2**len(Z))-1):
        IDsubsetZ_dec = i
        IDsubsetZ_bin = bin(IDsubsetZ_dec)
        subsetZ = getZsubset(IDsubsetZ_bin,Z)
        # no cache
        #print("from exist dseparator")
        dep = Dep(TargetNode,Xi,subsetZ, X, alpha)
        #print(subsetZ)
        #print(dep)
        if (dep==0):
            flagExist = True
            dsepSet = subsetZ
            break
    #print("Module exist d-separator: ",counter)
    return [flagExist,dsepSet]

In [5]:
def GetCondValues(z, condVars):
    if len(condVars)>1:
        maxz = 1
        for column in condVars:
            maxz*=int(column['name'].split('@')[1]) # equals to prod(sizes)
        d = np.zeros(len(condVars))
        
        div_dim = maxz/int(condVars[0]['name'].split('@')[1])
        num2div = z
        
        for i in range(0,len(condVars)-1):
            d[i] = floor(num2div/div_dim)
            num2div=num2div%div_dim
            div_dim = div_dim/int(condVars[i+1]['name'].split('@')[1])
        d[i] = num2div
    else:
        d = [z]
#     print("Size of condVars was: "+str(len(condVars)))
#     print(d)
#     print(sum([],d))
#     if (len(d)>1):
#         print(d)
#         print(z)
#         print_names(condVars)
    return d

In [6]:
def Dep(TargetNode, Xi, CondVars, X, alpha):
    #print('in Dep')
    global data_c_size
    
    vvTarget = []
    for i in range(int(TargetNode['name'].split('@')[1])):
        vvTarget.append(i)
    
    vvXi = []
    for i in range(int(Xi['name'].split('@')[1])):
        vvXi.append(i)
    
    szCondVars = 1
    #print(CondVars)
    for column in CondVars:
        #print(column)
        szCondVars *= int(column['name'].split('@')[1])
    
    if data_c_size <= (5 * len(vvTarget)*len(vvXi)*(szCondVars)):
        return 1
    
    if (len(CondVars)==0):
        S = np.zeros((len(vvTarget),len(vvXi)))
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                var1=np.array(TargetNode['data']) # Target Node
                var2=np.array(Xi['data'])
                # looking up for 0, 1 and 2
                #op1 = tf_mask_var(var1,vvTarget[i])
                op1 = np.array(list(map(lambda x: 1 if x==vvTarget[i] else 0, var1.tolist())))
                #op2 = tf_mask_var(var2,vvXi[j])
                op2 = np.array(list(map(lambda x: 1 if x==vvXi[j] else 0, var2.tolist())))
                #S[i][j] = tf_dot_mult(op1,op2)
                S[i][j]=np.sum(op1*op2)
                #print(S[i][j])
        G2=0
        N = np.sum(S)
#         print(S)
#         print(N)
        Si =np.sum(S,axis=1)
        Sj =np.sum(S,axis=0)
        Df = ((len(vvTarget)-1)*(len(vvXi)-1))
#         print(S[np.where(S>0)])
        Dedf = len((S[np.where(S>0)]))\
                      - len(Si[np.where(Si>0)])\
                      - len(Sj[np.where(Sj>0)]) +1
#         print("Dedf before if: ",Dedf)
        if (Dedf<1):
            Dedf=1
            #print('Here is the trouble')
            #return 0
        #print(S)
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                #print(i)
                #print(j)
                if (S[i][j]>0):
                    #print(Si[i])
                    #print(Sj[j])
                    G2 = G2 + S[i][j]*np.log((S[i][j])*N/(Si[i]*Sj[j]))
                    #print(G2)
        G2 = 2*G2
        
    else: # test conditional dependency
        
            
        S = np.zeros((len(vvTarget),len(vvXi),szCondVars))
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                for k in range(0,szCondVars):
                    condValue = GetCondValues(k,CondVars)
                    #print(condValue)
                    flagDataCondVars = np.ones(len(Xi['data']))
                    for l in range(0,len(CondVars)):
                        X_l = CondVars[l]
                        flagDataCondVars = flagDataCondVars*np.array(list(map(lambda x: 1 if x==condValue[l] else 0, X_l['data'])))
                        #op1 = tf_mask_var(X_l['data'],condValue[l])
                        #flagDataCondVars = np.array(list(map(lambda x: 1 if x==condValue[l] else 0, X_l.tolist())))
                        #flagDataCondVars = flagDataCondVars*op1
                    var1=np.array(TargetNode['data'])
                    var2=np.array(Xi['data'])
                    # looking up for -1, 0 and 1
                    op1 = np.array(list(map(lambda x: 1 if x==vvTarget[i] else 0, var1.tolist())))
                    #op1 = tf_mask_var(var1,vvTarget[i])
                    #print(op1)
                    #op2 = tf_mask_var(var2,vvXi[j])
                    op2 = np.array(list(map(lambda x: 1 if x==vvXi[j] else 0, var2.tolist())))
                    #print(op2)
                    #print(op1*op2*flagDataCondVars)
                    S[i][j][k]=np.sum(op1*op2*flagDataCondVars)
        G2 = 0
        
        Sjk = np.sum(S,axis=0)
        Sik = np.sum(S,axis=1)
        Sk = np.sum(Sjk,axis=0)
        
        #Sjk = sum(S,1);
        #Sik = sum(S,2);
        #Sk = sum(Sjk,2); 
        Dedf = len(S[np.where(S>0)]) -\
                len(Sik[np.where(Sik>0)]) -\
                len(Sjk[np.where(Sjk>0)]) +\
                len(Sk[np.where(Sk>0)])
        if Dedf<1:
            Dedf=1
            #print('No, here is the trouble')
            #return 0
        
        # compute G2 statistic
        #print(S.shape)
        #print(Sjk.shape)
        #print(Sik.shape)
        #print(Sk.shape)
        for i in range(0,len(vvTarget)):
            for j in range(0,len(vvXi)):
                for k in range(0,szCondVars):
                    if (S[i][j][k]>0):
                        #print(i,j,k)
                        #test_op =  Sk[k]
                        #test_op = Sik[i][k]
                        #test_op = Sjk[j][k]
                        G2=G2+(S[i][j][k] * np.log(\
                                    (S[i][j][k] * Sk[k])/\
                                    (Sik[i][k] * Sjk[j][k])))
        G2 = 2*G2
    
    assoc = (alpha - (1 - chi2.cdf(G2,Dedf)))/alpha
    #print(G2)
    #print(Dedf)
    #print(assoc)
    if assoc<0:
        assoc=0
#     print("Target Node: ",TargetNode['name'])
#     print("Node XI: ", Xi['name'])
#     print("\tG2:",G2)
#     print("\tDedf: ",Dedf)
#     print("\tAssoc: ",assoc)
#     print()
#     print()
    return assoc

In [7]:
def getZsubset(bin_id,Z):
    bin_str=str(bin_id[::-1])
    Zsubset=[]
    for i in range(0,len(bin_str)):
        if bin_str[i]=='1':
            Zsubset.append(Z[i])
    return Zsubset

In [8]:
def arrayUniverse(TargetNode,X):
    Universe = []
    for key in X:
        #print("Key in X:",key)
        #print("Target Node is: ",TargetNode)
        if (key!=TargetNode):
            append_dict={'name':key,'data':X[key].copy(deep=True).tolist()}
            Universe.append(append_dict)
    return Universe

In [9]:
def arrayX(X):
    returnable=[]
    for key in X:
        #tagged_c = MB_Column(X[key].copy(deep=True).tolist(),key)
        append_dict={'name':key,'data':X[key].copy(deep=True).tolist()}
        returnable.append(append_dict)
    return returnable

In [10]:
def print_names(dict_list):
    print()
    for item in dict_list:
        print(item['name'],end=" ")
    print()

In [11]:
def MinAssoc(TargetNode, Xi,Z, fixedCondVars, X, alpha):
    
    min_assoc=999
    #counter=0
    #print(fixedCondVars)
    if len(Z)==0:
        #print("from minassoc1")
        #print(fixedCondVars)
        min_assoc = Dep(TargetNode, Xi, fixedCondVars, X, alpha)
        #counter+=1
        subsetZ_min_assoc = fixedCondVars
        #print(min_assoc)
    else:
        #print(2**len(Z)-1)
        for IDsubsetZ_dec in range(0,2**len(Z)-1):
            IDsubsetZ_bin = bin(IDsubsetZ_dec)
            subsetZ = getZsubset(IDsubsetZ_bin,Z)
            #print(len(Xi))
            #print("from minassoc2")
            #print(IDsubsetZ_dec)
            #print(subsetZ)
            subsetZ_assoc=Dep(TargetNode, Xi, fixedCondVars+subsetZ,X,alpha)
            #counter+=1
            #print(subsetZ_assoc[IDsubsetZ_dec])
            if subsetZ_assoc < min_assoc:
                min_assoc = subsetZ_assoc
                subsetZ_min_assoc = fixedCondVars + subsetZ
                if (min_assoc==0):
                    break
    #print("Min Assoc module: ",counter)
    return min_assoc

In [12]:
def MaxMinHeuristic(TargetNode, CPC, Universe, X, alpha):
    F=[]
    assocF=-1
    Z = CPC
    fixedCondVars = []
    if (len(CPC)>0):
        Z = CPC[0:-1]           # all but the last one   
        fixedCondVars = [CPC[-1]] # we use last one
    for i in range(len(Universe)-1,-1,-1):
        if (len(Universe[i])==0):
            continue
        assoc = MinAssoc(TargetNode,Universe[i],Z,fixedCondVars,X,alpha)
        if (assoc>assocF):
            assocF = assoc
            F = Universe[i]
        if (assoc==0):
            Universe.pop(i)
#     value_dict=[]    
#     index_c = []
#     for i in range(0,len(Universe)):
#         index_c.append(i)
#     while len(value_dict)<len(Universe):
#         eval_index = random.randint(0,len(index_c)-1)
#         print("\nNode evaluating is: "+Universe[index_c[eval_index]]['name'])
#         value_dict.append([index_c[eval_index],MinAssoc(TargetNode,Universe[index_c[eval_index]], Z,fixedCondVars, X,alpha)])
#         index_c.pop(eval_index)
#         # Removing while looping is always best for performance
#         print("Resulting value was: "+str(value_dict[-1][1]))
#         if value_dict[-1][1] == 0:
#             # Remove node from Universe
#             last_eval = value_dict[-1][0]
#             Universe.pop(last_eval)
#             value_dict.pop(-1)
#             # Update index_c values
#             for k in range(0,len(index_c)):
#                 if index_c[k]>last_eval:
#                     index_c[k]=index_c[k]-1
#             # update value_dict
#             for k in range(0,len(value_dict)):
#                 if value_dict[k][0]>last_eval:
#                     value_dict[k][0]=value_dict[k][0]-1
    
#     for i in range(0,len(value_dict)):
#         if value_dict[i][1]>assocF:
#             assocF=value_dict[i][1]
#             F = Universe[value_dict[i][0]]
        #print(F)
    return [F,assocF,Universe]

In [13]:
def print_CPC(CPC):
    for i in CPC:
        print(i['name']+" ", end="")

In [14]:
def MMPC_(TargetNode,Universe,X,alpha):
    
    CPC=[]
    #print(TargetNode)
    # Phase I: Foward
    print("Entering Phase I")
    print("MMPC_beggining: \n"+str(len(Universe)))
    while len(Universe)>0:
        CPC_old = list(CPC) # copy
        maxminheur=MaxMinHeuristic(TargetNode,CPC,list(Universe),X,alpha)
        # maxminheur = [F, assocF, Universe]
        F = maxminheur[0]
        assocF = maxminheur[1]
        #print(assocF)
        Universe = maxminheur[2]
        #print("Universe printing line")
        #print(Universe)
        #print("\n=============================")
        #print(CPC)
        if assocF > 0:
            CPC.append(F)
            indF=Universe.index(F)
            Universe.pop(indF)
        #if (len(CPC)==len(CPC_old)) or (len(CPC)>0.3*(len(Universe)-1)):
        if (len(CPC)==len(CPC_old)):
            break
        print("\nUniverse actual size:")
        print(len(Universe))
        print("CPC actual size:")
        print(len(CPC))
        print("CPC contents:")
        print_CPC(CPC)
    
    # Phase 2: Backward
    print("\nEntering Phase II")
    CPC=CPC[::-1]
    if len(CPC)>1:
        Z=list(CPC)
        for i in range(len(CPC)-1,-1,-1):
            # index is i
            Z.pop(i)
            #print("Analyzing D-separator for ",CPC[i]['name'])
            if ExistDseparator(TargetNode,CPC[i],Z,X,alpha)[0] == True:
                #print("it did exist! removing from cpc.")
                CPC.pop(i)
#     return symmetry_test_v2(TargetNode,CPC,X,alpha)
    return CPC

In [15]:
def MMPC(TargetNode, X, alpha):
    # The universe will be an array of DataFrame Columns
    Universe = arrayUniverse(TargetNode,X)
    print(len(Universe))
    X = arrayX(X)
    print(len(X))
    for column in X:
        if (column['name']==TargetNode):
            TargetNode = column
            break
    CPC = MMPC_(TargetNode,Universe,X,alpha)
    return CPC

In [16]:
def MMPC_Parallel(arg):
    TargetNode, X, alpha = arg
    Universe = arrayUniverse(TargetNode,X)
    print(len(Universe))
    X = arrayX(X)
    print(len(X))
    for column in X:
        if (column['name']==TargetNode):
            TargetNode = column
            break
    CPC = MMPC_(TargetNode,Universe,X,alpha)
    return CPC

In [17]:
def getKeysPartition(targetNode, data_keys,n_partitions):
    n_partitions = int(np.log2(len(data_keys))/2)  # log2 ( number of variables ) divided by 2
    partitions = []
    partition_diff = int(int(len(data_keys)/n_partitions)/2)
    for i in range(0,n_partitions):
        if (i==0):
            partitions.append(list(data_keys[-partition_diff::])+list(data_keys[0:int(len(data_keys)/n_partitions)]))
            continue
        if (i<n_partitions-1):
            partitions.append(list(data_keys[-partition_diff+int(len(data_keys)/n_partitions)*i:int(len(data_keys)/n_partitions)*(i+1)]))
            continue
        if (i==n_partitions-1):
            partitions.append(list(data_keys[-partition_diff+int(len(data_keys)/n_partitions)*i::]))
            
    for i in range(0,n_partitions):
        for j in range(len(partitions[i])-1,-1,-1):
            if (partitions[i][j]==targetNode):
                partitions[i].pop(j)
        partitions[i] = partitions[i] + [targetNode]
    return partitions,n_partitions

In [ ]:
type(keys_partitions[0][0][0][0])

In [26]:
%%time
from sklearn.metrics import precision_recall_fscore_support as full_score
alpha=0.05
# Initial vectors setup
#const_vectors = setup_const_vectors(data_vars,data_c_size)
#for i in range(0,len(X.keys())):    
#for i in range(0,1):  
print("======================================================================================")

TargetNode =X.keys()[0]
PC=MMPC(TargetNode,X,alpha)
print("Target Node is: ",TargetNode)
print("PC set is :")
for j in PC:
    print(j['name'])

TargetNodeDict={}
TargetNodeDict['name'] = TargetNode
TargetNodeDict['data'] = X[TargetNode]
clf = LinearSVC(random_state=0)
X_test=[]
Y_test=[]
for i in range(0,data_c_size):
    d_row = []
    for pc_column in PC:
        d_row.append(pc_column['data'][i])
    X_test.append(d_row)
for i in range(0,data_c_size):
    Y_test.append(TargetNodeDict['data'][i])
clf.fit(X_test[:-1000],Y_test[:-1000])
full_score(Y_test[-1000:],clf.predict(X_test[-1000:]))

36
37
Entering Phase I
MMPC_beggining: 
36

Universe actual size:
19
CPC actual size:
1
CPC contents:
Node29@3 
Universe actual size:
18
CPC actual size:
2
CPC contents:
Node29@3 Node27@2 
Universe actual size:
17
CPC actual size:
3
CPC contents:
Node29@3 Node27@2 Node26@3 
Universe actual size:
15
CPC actual size:
4
CPC contents:
Node29@3 Node27@2 Node26@3 Node22@4 
Entering Phase II
Target Node is:  Node1@4
PC set is :
Node22@4
Node26@3
CPU times: user 2min 9s, sys: 0 ns, total: 2min 9s
Wall time: 2min 15s


/home/a20114261/anaconda3/envs/GeneInteractionsBN/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [29]:
TargetNodeDict={}
TargetNodeDict['name'] = TargetNode
TargetNodeDict['data'] = X[TargetNode]
clf = LinearSVC(random_state=0)
X_test=[]
Y_test=[]
for i in range(0,data_c_size):
    d_row = []
    for pc_column in PC:
        d_row.append(pc_column['data'][i])
    X_test.append(d_row)
for i in range(0,data_c_size):
    Y_test.append(TargetNodeDict['data'][i])
clf.fit(X_test[:-1000],Y_test[:-1000])
precision, recall, fscore, support=full_score(Y_test[-1000:],clf.predict(X_test[-1000:]))
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

precision: [ 0.97        0.          0.86636971  0.        ]
recall: [ 0.85840708  0.          0.9974359   0.        ]
fscore: [ 0.91079812  0.          0.9272944   0.        ]
support: [113  65 780  42]


/home/a20114261/anaconda3/envs/GeneInteractionsBN/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
from joblib import Parallel, delayed

In [58]:
X_graph = pd.read_csv('./Alarm1_graph.txt',delimiter='  ',header=None)

/home/a20114261/anaconda3/envs/GeneInteractionsBN/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [ ]:
X_graph = X_graph.transpose()

In [59]:
def related_nodes(X_graph,X,ind):
    print("Acoording to the true graph, the PC set for ",X.keys()[ind],"is:")
    for i in range(0,len(X_graph[ind])):
        if (X_graph[ind][i]==1):
            print(X.keys()[i])
    X_graph = X_graph.transpose()
    for i in range(0,len(X_graph[ind])):
        if (X_graph[ind][i]==1):
            print(X.keys()[i])
    X_graph = X_graph.transpose()

In [60]:
for i in range(0,len(X_graph.keys())):    
    print("====================================================")
    related_nodes(X_graph,X,i)

Acoording to the true graph, the PC set for  Node1@4 is:
Node22@4
Node29@3
Acoording to the true graph, the PC set for  Node2@4 is:
Node23@4
Node27@2
Node29@3
Acoording to the true graph, the PC set for  Node3@3 is:
Node30@2
Acoording to the true graph, the PC set for  Node4@4 is:
Node15@3
Node22@4
Acoording to the true graph, the PC set for  Node5@3 is:
Node6@2
Node13@3
Acoording to the true graph, the PC set for  Node6@2 is:
Node5@3
Acoording to the true graph, the PC set for  Node7@3 is:
Node9@2
Node13@3
Acoording to the true graph, the PC set for  Node8@3 is:
Node9@2
Node13@3
Acoording to the true graph, the PC set for  Node9@2 is:
Node7@3
Node8@3
Acoording to the true graph, the PC set for  Node10@3 is:
Node12@3
Node16@3
Acoording to the true graph, the PC set for  Node11@2 is:
Node36@2
Acoording to the true graph, the PC set for  Node12@3 is:
Node13@3
Node34@3
Node10@3
Acoording to the true graph, the PC set for  Node13@3 is:
Node14@2
Node5@3
Node7@3
Node8@3
Node12@3
Acoording to

In [ ]:
#Results for Node1 should { 2, 22, 23, 25 } 

In [234]:
from sklearn.svm import LinearSVC


0.956


In [140]:
a=1

In [141]:
a

1

In [142]:
import random

9

In [160]:
a=[1,2,3,4,5]

5

[4, 3]

In [184]:
a

[1, 2, 3, 4]

In [224]:
a[:-1]

[1, 2, 3]

[1, 3]

In [230]:
a[-1:]

[4]